## Generate MetaCyc-dependendt files

### protein_complexes.txt

In [1]:
import cobra
import os
import pandas as pd
import numpy as np
import urllib
import pythoncyc as pc

## Metabolites

met_output_file = 'metabolites.txt'
m_model_file = './iYO844.json'

m_model = cobra.io.load_json_model(m_model_file)

## Fix of gene reaction rules
m_model.reactions.ACCOAC.gene_reaction_rule = '(BSU29200 and BSU29210) or BSU24350 or BSU24340 or BSU22440'
m_model.reactions.PDH.gene_reaction_rule = '(BSU14580 and BSU14590) or BSU14600 or BSU14610'

In [2]:
bsub = pc.select_organism('bsub')

In [3]:
CPLXS = pc.PGDB.all_protein_complexes(bsub,filter='all')

In [4]:
filename = 'protein_complexes.txt'
file = open(filename,'w')
CPLX_list = []
grRule_list = []

for cplx in CPLXS:
    
    cplx_string = str(cplx)
    cplx_string = cplx_string[1:len(cplx_string)-1]
    string = cplx_string + '\t' + 'default_name' + '\t'
    grRule_string = ''
    
    genes_of_cplx = pc.PGDB.genes_of_protein(bsub,cplx)
    
    for gene in genes_of_cplx:
        gene_string = str(gene)
        gene_string = gene_string[1:len(gene_string)-1]
        string = string + gene_string +'(1)' + ' AND '
        grRule_string = grRule_string + gene_string + ' and '
        
    string = string[0:len(string)-5]
    string = string + '\t' + 'M_protein_recon' + '\n'
    file.write(string)
    
    grRule_string = grRule_string[0:len(grRule_string)-5]
    grRule_list.append(grRule_string)
    CPLX_list.append(cplx_string)
    
file.close()

### enzyme_reaction_association.txt

In [17]:
filename = 'enzyme_reaction_association_test.txt'
file = open(filename,'w')

rule_list = []
enz_rxn_assoc_list = []
for reaction in m_model.reactions:
    string = ''
    
    rule_string = str(reaction.gene_reaction_rule)
    
    rule_string = rule_string.replace('(','')
    rule_string = rule_string.replace(')','')
    
    rule_list = rule_string.split(' or ')
    
    print rule_list
    
    enz_rxn_assoc = []
    for rule in rule_list:
        try:
            index = grRule_list.index(rule)
            enz_rxn_assoc.append(CPLX_list[index])
        except:
            if rule_string:
                if len(rule) == 8:
                    enz_rxn_assoc.append(rule + '-MONOMER')
                else:
                    enz_rxn_assoc.append(rule)
            else:
                enz_rxn_assoc.append('EMPTY-RULE')
    enz_rxn_assoc_list.append(enz_rxn_assoc)
    
    string = str(reaction.id) + '\t'
    for item in enz_rxn_assoc:
        string = string + item + ' OR '
        
    string = string[0:len(string)-4]
    string = string + '\n'
    file.write(string)
file.close()

['BSU07840']
['BSU07840']
['BSU07840']
['BSU07840']
['BSU16730 and BSU16740']
['']
['']
['']
['BSU30820']
['BSU06310']
['']
['']
['']
['BSU07840']
['BSU07840']
['BSU07840']
['BSU07840']
['']
['']
['']
['']
['BSU03040']
['BSU38560', 'BSU40110']
['BSU03900']
['']
['BSU19310', 'BSU38830', 'BSU39860']
['BSU32830']
['BSU24170']
['BSU32830']
['BSU32830']
['BSU32830']
['BSU32830']
['BSU32830']
['BSU32830']
['']
['BSU29200 and BSU29210', 'BSU24350', 'BSU24340', 'BSU22440']
['BSU07700 and BSU13900 and BSU13910']
['BSU11210']
['BSU11200']
['BSU36010', 'BSU28310 and BSU28300']
['BSU29470']
['BSU36000']
['BSU28310 and BSU28300', 'BSU36010']
['']
['']
['']
['BSU32820', 'BSU24150', 'BSU37170', 'BSU18260', 'BSU04520']
['BSU04520', 'BSU18260', 'BSU32820']
['BSU37170', 'BSU24150']
['BSU18260', 'BSU32820', 'BSU04520']
['BSU24150', 'BSU37170']
['BSU37170', 'BSU24150']
['BSU24150', 'BSU37170']
['BSU37170', 'BSU24150']
['BSU24150', 'BSU37170']
['BSU15350']
['BSU18000']
['BSU11220']
['BSU04620']
['BSU19230'

['BSU06480 and BSU06470 and BSU06460']
['BSU32440', 'BSU32430']
['BSU34880']
['BSU03200']
['BSU02980 and BSU02990 and BSU03000', 'BSU33830 and BSU33820 and BSU33810 and BSU33800']
['BSU06660', 'BSU30530', 'BSU03220', 'BSU19580']
['BSU00510']
['BSU22600']
['BSU02290']
['BSU10020']
['']
['']
['BSU04740']
['BSU04740']
['BSU37660']
['BSU37660']
['BSU23760']
['BSU13730']
['BSU15020']
['BSU23490', 'BSU19630']
['BSU19630', 'BSU23490']
['BSU23490', 'BSU19630']
['BSU19630', 'BSU23490']
['BSU23490', 'BSU19630']
['BSU19630', 'BSU23490']
['BSU23490', 'BSU19630']
['BSU23490', 'BSU19630']
['BSU03070']
['BSU29180']
['BSU39400']
['BSU39400']
['BSU00110 and BSU00120']
['']
['BSU39980']
['BSU27850']
['']
['BSU29300', 'BSU16670']
['BSU23270 and BSU23250']
['BSU23270 and BSU23250']
['BSU28790']
['BSU35920']
['BSU28790']
['BSU28780']
['']
['']
['BSU35940 and BSU35960 and BSU35950 and BSU35930']
['BSU35940 and BSU35960 and BSU35950 and BSU35930']
['BSU23490', 'BSU19630']
['BSU31180']
['BSU31200']
['']
['']


In [16]:
m_model.reactions.ACt2r.gene_reaction_rule

u''